In [1]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer

sns.set_theme()

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
terms = [
    'bueno',       
    'bien',        
    'aceptable',   
    'adecuado',    
    'virtuoso',    
    'justo',       
    'está bien',   
    'genial',      
    'increíble',   
    'excepcional', 
    'significativo', 
    'mejor',       
    'ok',           
    'bien',         
    'malo',         
    'horrible',     
    'peor',         
    'terrible',     
    'horrible',
]


In [ ]:
class sentence_similarity():

  def __init__(self, model, sentencias, n_componentes=2):
    self.model = model
    self.pca=PCA(n_components=n_componentes)
    self.embeddings = self.model.encode(sentencias)
    self.pca = self.pca.fit(self.embeddings)
    self.component = self.pca.fit_transform(self.embeddings)

  def varianza_explicada(self):
    return self.pca.explained_variance_

  def componentes(self):
    return self.component

  def k_means(self,clusters):
    return KMeans(n_clusters=clusters).fit(self.component)

class graficos():

  def planos_factoriales(self):
    pass

  def planos_kmeans(self):
    pass
  
  def worcloud(self):
    pass


class limpieza_regex():
  def __init__(self,sentencias):
    self.datos=sentencias

  def limpieza_componentes(self):
    #elimina numeros

    # elimina caracteres extraños

    #eliminar espacios dobles

    pass 

  def stop_words():
    





In [ ]:




model = SentenceTransformer('hackathon-pln-es/paraphrase-spanish-distilroberta')
modelos = sentence_similarity(model,terms,4)

modelos.varianza_explicada()
modelos.componentes()

array([[ 3.7494905 , -2.2132826 , -0.1643261 , -0.3805743 ],
       [ 6.220342  , -3.4253685 , -0.6411597 , -1.2827947 ],
       [ 1.9704607 ,  1.9698063 ,  0.5390244 ,  0.04644234],
       [ 1.768573  ,  3.1530187 ,  1.4846843 ,  1.5600691 ],
       [ 0.01688309,  2.1217403 , -0.01583855,  2.4510436 ],
       [ 3.6733449 , -0.4909285 ,  0.58783674, -1.4102707 ],
       [ 5.2500305 , -1.9660449 , -1.1663163 , -0.7918496 ],
       [ 0.47052917,  2.77278   ,  0.4252021 ,  2.7935534 ],
       [-1.5312409 ,  0.19315939, -7.487824  ,  1.4925495 ],
       [-1.94183   ,  8.38754   , -0.85627156, -6.530071  ],
       [ 1.5908741 ,  4.9567986 ,  1.5266852 ,  4.261724  ],
       [ 1.211852  , -0.1892257 ,  1.4594605 ,  0.73906344],
       [ 3.0275495 , -1.3942894 ,  0.8572375 , -0.2636167 ],
       [ 6.220342  , -3.4253702 , -0.6411606 , -1.2827951 ],
       [-4.265207  , -2.113771  ,  2.7977018 , -1.211612  ],
       [-7.3267026 , -1.9544387 , -1.0109322 ,  0.46748582],
       [-5.347252  , -2.

In [ ]:
cluster = modelos.k_means(4)
cluster.labels_

array([0, 0, 2, 2, 2, 0, 0, 2, 2, 3, 2, 2, 0, 0, 1, 1, 1, 1, 1],
      dtype=int32)

In [ ]:
cluster.cluster_centers_[0]

array([ 4.6901836 , -2.1525474 , -0.19464807, -0.9019835 ], dtype=float32)

In [ ]:
from textblob import TextBlob
from textblob.exceptions import NotTranslated

textos = [
    "Este producto es increíblemente bueno y maravilloso.",
    "La experiencia fue muy mala y desagradable.",
    "No estoy convencido de que sea genial, pero tampoco es terrible."
]

for texto in textos:
    blob = TextBlob(texto)
    try:
        polaridad = blob.translate(from_lang='es', to='en').sentiment.polarity
    except NotTranslated:
        polaridad = blob.sentiment.polarity

    sentimiento = "Positivo" if polaridad > 0 else "Negativo" if polaridad < 0 else "Neutro"
    print(f"Texto: {texto}")
    print(f"Polaridad: {polaridad:.2f}, Sentimiento: {sentimiento}")
    print()


Texto: Este producto es increíblemente bueno y maravilloso.
Polaridad: 0.85, Sentimiento: Positivo

Texto: La experiencia fue muy mala y desagradable.
Polaridad: -0.78, Sentimiento: Negativo

Texto: No estoy convencido de que sea genial, pero tampoco es terrible.
Polaridad: 0.65, Sentimiento: Positivo



In [ ]:
def plot_palabras (self,sentencias):
    coordenadas = self.__entrenamiento_PCA(self.__enconde(sentencias))
    x = coordenadas[:,0]
    y = coordenadas[:,1]
    fig, ax = plt.subplots(figsize=(15, 6))
    ax.scatter(x, y, c='b')

    for i, term in enumerate(terms):
        ax.annotate(
            term,
            (x[i], y[i]),
            xytext=(4, 2),
            textcoords='offset points'
        )

    plt.title('word embeddings')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()